# Working With TileMatrixSets (other than WebMercator)


[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/master?filepath=docs%2Fexamples%2FWorking_with_nonWebMercatorTMS.ipynb)

TiTiler has builtin support for serving tiles in multiple Projections by using [rio-tiler](https://github.com/cogeotiff/rio-tiler) and [morecantile](https://github.com/developmentseed/morecantile).

The default `cog` and `stac` endpoint (`titiler.endpoints.cog`and `titiler.endoints.stac`) are built with Mutli TMS support using the default grids provided by morecantile:

```python

from fastapi import FastAPI

from titiler.endpoints.factory import TilerFactory

# Create a Multi TMS Tiler using `TilerFactory` Factory
cog = TilerFactory(router_prefix="cog")

app = FastAPI()
app.include_router(cog.router, prefix="/cog", tags=["Cloud Optimized GeoTIFF"])

```

This Notebook shows how to use and display tiles with non-webmercator TileMatrixSet

#### Requirements
- ipyleaflet
- requests

In [1]:
# Uncomment if you need to install those module within the notebook
# !pip install ipyleaflet requests

In [2]:
import json

import requests

from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    WMSLayer,
    GeoJSON,
    projections
)

In [3]:
titiler_endpoint = "https://api.cogeo.xyz/"  # Devseed Custom TiTiler endpoint
url = "https://s3.amazonaws.com/opendata.remotepixel.ca/cogs/natural_earth/world.tif" # Natural Earth WORLD tif

### List Supported TileMatrixSets

In [4]:
r = requests.get("https://api.cogeo.xyz/tileMatrixSets").json()

print("Supported TMS:")
for tms in r["tileMatrixSets"]:
    print("-", tms["id"])

Supported TMS:
- CanadianNAD83_LCC
- EPSG3413
- EPSG6933
- EuropeanETRS89_LAEAQuad
- LINZAntarticaMapTilegrid
- NZTM2000
- UPSAntarcticWGS84Quad
- UPSArcticWGS84Quad
- UTM31WGS84Quad
- WebMercatorQuad
- WorldCRS84Quad
- WorldMercatorWGS84Quad


## WGS 84 -- WGS84 - World Geodetic System 1984 - EPSG:4326

https://epsg.io/4326

In [51]:
r = requests.get(
    "https://api.cogeo.xyz/cog/WorldCRS84Quad/tilejson.json", params = {"url": url}
).json()

m = Map(center=(45, 0), zoom=4, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(url=r["tiles"][0], opacity=1)
m.add_layer(layer)
m

Map(center=[45, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## WGS 84 / NSIDC Sea Ice Polar Stereographic North - EPSG:3413

https://epsg.io/3413

In [52]:
r = requests.get(
    "https://api.cogeo.xyz/cog/EPSG3413/tilejson.json", params = {"url": url}
).json()

m = Map(center=(70, 0), zoom=1, basemap={}, crs=projections.EPSG3413)

layer = TileLayer(url=r["tiles"][0], opacity=1)
m.add_layer(layer)
m

Map(center=[70, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## ETRS89-extended / LAEA Europe - EPSG:3035

https://epsg.io/3035

In [53]:
r = requests.get(
    "https://api.cogeo.xyz/cog/EuropeanETRS89_LAEAQuad/tilejson.json", params = {"url": url}
).json()

my_projection = {
    'name': 'EPSG:3035',
    'custom': True, #This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    'proj4def': '+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs',
    'origin': [6500000.0, 5500000.0],
    'resolutions': [
        8192.0,
        4096.0,
        2048.0,
        1024.0,
        512.0,
        256.0
    ]
}

m = Map(center=(50, 65), zoom=1, basemap={}, crs=my_projection)

layer = TileLayer(url=r["tiles"][0], opacity=1)
m.add_layer(layer)
m

Map(center=[50, 65], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…